In [2]:
import json
import os
import logging
import pandas as pd
from dotenv import load_dotenv
from typing import Dict
from pprint import pprint
from pathlib import Path
from llm_assessor import *
from langchain.chat_models import ChatOpenAI

load_dotenv(override=True)

True

In [3]:
# Set working directory - Not required if using Jupyter outside of VScode
workdir = os.environ["workdir"]
os.chdir(workdir)
os.getcwd()

'/Users/junaid/Developer/audenshaw_exam_validation'

In [4]:
logging.basicConfig(
    level=logging.INFO,
    format='%(process)d-%(levelname)s-%(message)s'
    )


# Restructure AQA History Mark Scheme

In [5]:
hist_mark_scheme = ["""
Level 2: Developed analysis of source based on content and/or provenance 3–4
Students may progress from a simple analysis of the source with extended
reasoning supported by factual knowledge and understanding related to the
features of the source.
For example, the cartoon is critical of the League of Nations because it was
produced at the time of the Manchurian Crisis when the League failed to stop
the Japanese invasion of Manchuria. The title of the cartoon suggests that Japan
is ignoring the League of Nations.
Level 1: Simple analysis of source based on content and/or provenance 1–2
Students identify relevant features in the source and support them with simple
factual knowledge and understanding.
For example, the cartoon shows that Japan is walking all over the League of
Nations.
Students either submit no evidence or fail to address the question 0
""",
"""
Level 2: Developed analysis of source based on content and/or provenance
Students may progress from a simple analysis of the source with extended
reasoning supported by factual knowledge and understanding related to the
features of the source.
For example, the cartoon is British and the title refers to being betrayed by
Russia who was one of their allies. The Bolshevik Revolution occurred in 1917
and Lenin made peace with Germany in 1918. This reduced the demands on
the German army in the East and allowed them to concentrate all their forces
against the Allies on the Western Front.
3–4
Level 1: Simple analysis of source based on content and/or provenance
Students identify relevant features in the source and support them with simple
factual knowledge and understanding.
For example, the Bolshevik is dragging Russia towards Germany against her
will; the cartoon was published in the year Russia withdrew from the First
World War.
1–2
Students either submit no evidence or fail to address the question 0
""",
"""
Target Analyse sources contemporary to the period (AO3a)
Evaluate sources and make substantiated judgements (AO3b)
In analysing and evaluating sources, students will draw on their contextual
knowledge to question critically the content and provenance of sources (for
example, the context of the time in which source was created, place, author’s
situation, knowledge, beliefs, circumstances, access to information, purpose
and audience).
Level 4: Complex evaluation of both sources with sustained judgement 10–12
based on content and provenance
Students may progress from a developed evaluation of the sources by complex
reasoning related to utility on the basis of content and provenance. They may
evaluate the relationship between the sources based on analysis of provenance
and contextual knowledge.
For example, the sources are useful to show how difficult it was for the
governments of the time to respond to the threats posed by Hitler. The Munich
Agreement mentioned in Source B was the work of several countries, not just
Britain alone, therefore the responsibility for giving in to his demands is a
shared one. The American view in Source C is useful to show that even countries
outside Europe were concerned with the issue of Czechoslovakia and the policy
of appeasement.
Level 3: Developed evaluation of sources based on the content and/or 7-9
provenance
Students may progress from a simple evaluation of the sources with extended
reasoning related to utility on the basis of content and/or provenance.
For example, Source B is useful for showing a positive view of appeasement as
Chamberlain presented it to the people of Britain. The Munich Agreement
allowed Hitler to occupy the Sudetenland in Czechoslovakia despite it not being
German land. However, in return for this concession, Chamberlain secured the
Anglo-German declaration which was a promise that war would be avoided.
Source C is critical of Chamberlain for thinking that Hitler will be content with
the concessions. The Munich Agreement had allowed Hitler to take over land in
Czechoslovakia. Hitler had promised not to make any further demands for more
territory but many people at the time thought he would try and take more land.
For example, Source C is useful because it shows an opinion from outside
Europe that was critical of Chamberlain’s policy of appeasement. The cartoon
reflects the view that by giving into Hitler the Munich Agreement would
encourage his territorial ambitions even further. This prediction came true
when Hitler invaded the rest of Czechoslovakia in 1939. Source B shows why, in
the short term, appeasement seemed to have worked. Chamberlain wanted to
use discussion to avoid war with Hitler and Source B was seen as evidence that
he had achieved his aim.
Level 2: Simple evaluation of source(s) based on content and/or provenance 4–6
Students may progress from a basic analysis of the source(s) to simple
evaluation of the content and/or provenance.
For example, Source B shows that appeasement seemed to have worked.
Chamberlain wanted to use discussion to avoid war with Hitler and this joint
statement was seen as evidence that he had achieved his aim.
For example, Source C is criticising appeasement. The American cartoon is
saying the Munich Agreement might not keep Hitler quiet for long. The Munich
Agreement had allowed Hitler to take over land in Czechoslovakia. Hitler had
promised not to make any further demands for more territory
Level 1: Basic analysis of sources(s) 1–3
Answers may show understanding/support for one or both sources, but the
case is made by assertion/basic inference.
Students identify basic features which are valid about the sources and related to
the enquiry point, for example, Source B says that Germany and Britain agreed
never to go to war.
Source C shows that Chamberlain is trying to keep Hitler quiet.
Students either submit no evidence or fail to address the question 0
""",
"""
Level 4: Complex analysis of causation/consequence 7–8
Answer is presented in a coherent narrative/account that demonstrates a
range of accurate and detailed knowledge and understanding that is
relevant to the question
Extends Level 3.
Students may progress from a developed narrative of causation/consequence
with complex sequencing and reasoning supported by a range of accurate and
detailed factual knowledge and understanding which might be related, for
example, to an analysis of how/why tension changed at different stages and/or
showing understanding about how much each part of the sequence reduced
tension.
For example, the contribution to international peace by the League sometimes
depended on the status of the country they were defending. In the issue of the
Corfu invasion by Mussolini, the League originally upheld the complaint by
Greece but this was undermined by the influence of more powerful countries
such as Britain and France who chose to support Italy’s interests.
Level 3: Developed analysis of causation/consequence 5–6
Answer is presented in a structured and well-ordered narrative/account
that demonstrates a range of accurate knowledge and understanding that
is relevant to the question.
Extends Level 2.
Students may progress from a simple narrative of causation/consequence with
developed sequencing and reasoning supported by a range of accurate factual
knowledge and understanding which might be related, for example to an
analysis of how/why tension was reduced at one stage in the process.
For example, the League contributed to peace because it helped to avoid a war
breaking out between Sweden and Finland. Both countries wanted to claim
ownership of the Aaland Islands. The League found a compromise that both
countries accepted.
Level 2: Simple analysis of causation/consequence 3–4
Answer is presented in a structured account that demonstrates specific
knowledge and understanding that is relevant to the question
Students may progress from a basic narrative of causation/consequence by
showing a simple understanding of sequencing, supporting it with factual
knowledge and understanding.
For example, the League of Nations created an international court of justice to
settle disagreements between countries. Every member of the Assembly could
vote to choose the judges.
Level 1: Basic analysis of causation/consequence 1–2
Answer is presented as general statements which demonstrates basic
knowledge and understanding that is relevant to the question
Students identify cause(s)/consequence(s) about the events such as the League
tried to prevent another war by discussing problems. The Commission for
Refugees helped prisoners of war to return to their homes.
Students either submit no evidence or fail to address the question 0
""",
"""
Level 4: Complex explanation of stated factor and other factor(s) leading to 13–16
a sustained judgement
Answer demonstrates a range of accurate and detailed knowledge and
understanding that is relevant to the question
Answer demonstrates a complex, sustained line of reasoning which has a
sharply-focused coherence and logical structure that is fully substantiated, with
well-judged relevance.
Extends Level 3.
Students may progress from a developed explanation of causation by complex
explanation of the relationship between causes supported by detailed factual
knowledge and understanding to form a sustained judgement.
For example, the loss of territory was not the main reason because Germany
expected some losses. What they hated the most about the Treaty of Versailles
was that Germany was excluded from the negotiations and it was a ‘Diktat’. This
meant they could not influence the severity of the resulting terms. This meant
they could not prevent the inclusion of Clause 231 concerning war guilt which
said they were responsible for starting the war.
Level 3: Developed explanation of the stated factor and other factor(s) 9-12
Answer demonstrates a range of accurate knowledge and understanding
that is relevant to the question
Answer demonstrates a developed, sustained line of reasoning which has
coherence and logical structure; it is well substantiated, and with sustained,
explicit relevance.
Extends Level 2.
Answer may suggest that one reason has greater merit.
Students may progress from a simple explanation of causation with developed
reasoning supported by factual knowledge and understanding.
For example, the territorial terms of the Treaty of Versailles were the main
reason why Germany hated it because they were humiliating and economically
damaging. 10% of its land was given to neighbouring countries. A strip of land
was given to Poland which effectively split Germany in two by creating a corridor
that created access to the sea. As well as losing land outright, Germany also lost
control of areas such as the coal rich Saarland which was put under a League of
Nations mandate for 15 years and the Rhineland was demilitarised.
For example, the reparations payments were another reason why Germans
hated the Treaty of Versailles because it would make it harder for Germany to
recover. People were suffering from starvation after the war, and they were
angry that the reparation bill was so huge. Germany had to pay money to the
Allies for the damage caused during the war. It was estimated that it would take
over 60 years to pay back the full bill.
Level 2: Simple explanation of stated factor or other factor(s) 5-8
Answer demonstrates specific knowledge and understanding that is
relevant to the question
Answer demonstrates a simple, sustained line of reasoning which is coherent,
structured, substantiated and explicitly relevant.
Answers arguing a preference for one judgement but with only basic
explanation of another view will be marked at this level.
Students may progress from a basic explanation of causation by simple
reasoning and supporting it with factual knowledge and understanding.
For example, the War Guilt clause was hated by the German people because it
forced them to accept the full blame for starting the war. Germany had no
choice about signing the treaty; they were threatened with renewed fighting if
they did not accept all the terms.
Level 1: Basic explanation of one or more factors 1–4
Answer demonstrates basic knowledge and understanding that is relevant
to the question
Answer demonstrates a basic line of reasoning, which is coherent, structured
with some substantiation; the relevance might be implicit.
Students recognise and provide a basic explanation of one or more factors.
Students may offer a basic explanation of the stated factor, such as the Treaty
of Versailles took a lot of land from Germany.
Students may offer basic explanations of other factor(s), for example, Germany
was humiliated and forced to reduce the size of its army.
Students either submit no evidence or fail to address the question 0
"""

]

In [6]:
structure_ms_prompt = """
Format the Mark Scheme Text supplied below according to the following YAML structure:

Level_number:
  description: 
    - Description of Marking Level
  marks_awarded:
    maximum: int
    minimum: int
  criteria:
    - Make a list for each criteria that earns a mark
  progression: 
    - If relevant, how can student progress from previous level to current level
  example_answers: 
    -

Do not invent anything, extract and format the data only. If there is no information to fall into a YAML section, do not include that section.

Mark Scheme Text:
{mark_scheme_text}
"""

llm=ChatOpenAI(model_name="gpt-4o")

response = run_prompt_chain(
  prompt_template=structure_ms_prompt,
  llm=llm,
  mark_scheme_text=hist_mark_scheme[4]
)

In [8]:
print(response["prompt_chain_response"])

```yaml
Level_4:
  description: 
    - Complex explanation of stated factor and other factor(s) leading to a sustained judgement
  marks_awarded:
    maximum: 16
    minimum: 13
  criteria:
    - Answer demonstrates a range of accurate and detailed knowledge and understanding that is relevant to the question
    - Answer demonstrates a complex, sustained line of reasoning which has a sharply-focused coherence and logical structure that is fully substantiated, with well-judged relevance.
  progression: 
    - Extends Level 3.
    - Students may progress from a developed explanation of causation by complex explanation of the relationship between causes supported by detailed factual knowledge and understanding to form a sustained judgement.
  example_answers: 
    - The loss of territory was not the main reason because Germany expected some losses. What they hated the most about the Treaty of Versailles was that Germany was excluded from the negotiations and it was a ‘Diktat’. This meant 

In [10]:
# Cost of restructuring
print(response["prompt_chain_token_costing"]["total_cost"])

0.018885


# Restructure Edexcel Business Studies Mark Scheme

In [11]:
edexcel_business_studies_mark_scheme_texts = [
"""
Award 1 mark for identification of a way, plus 2 further marks for
explaining this way up to a total of 3 marks.
The business could choose a cheaper location (1). This is because
the business can use e-commerce (1). As a result, the business
may not require a physical retail location (1).
The business will need to be in a location with good internet
connections (1). This is because a business may need the internet
to communicate with stakeholders (1). A location with a weak
signal could cause barriers to communication for the business (1).
Accept any other appropriate response. Answers that list more
than one way with no explanation will be awarded a maximum of
1 mark.""",
"""
Award 1 mark for identification of a way, plus 2 further marks for
explaining this way up to a total of 3 marks.
A small business could gain an advantage by charging lower prices
(1). This would mean customers are more likely to buy the
product because it is cheaper than other products (1). Therefore,
this would attract customers from its competitors as they want to
save money (1).
A small business could improve the quality of the product (1). This
would attract customers because they think they are getting a
better product (1). As a result, the business would gain a better
reputation than competitors (1).
Accept any other appropriate response. Answers that list more
than one way with no explanation will be awarded a maximum of
1 mark.""",
"""
Award 1 mark for identification of an advantage, plus 2 further
marks for explaining this advantage up to a total of 3 marks.
A small business would not get sued for breaking the law (1). This
means the business would not have to face financial consequences
such as a fine (1). This will prevent the business from having a
negative image (1).
Employees will feel that the business is taking good care of them
(1). Therefore, the business will have a more motivated workforce
(1). This could lead to an increase in productivity (1).
Accept any other appropriate response. Answers that list more
than one advantage with no explanation will be awarded a
maximum of 1 mark.""",
"""
Award 1 mark for identification of a disadvantage, plus 2 further
marks for explaining this disadvantage, up to a total of 3 marks.
One disadvantage is that the employees will want to leave the
business (1). This means the business may have to find new
employees (1). This may increase the recruitment costs of the
business (1).
The employees may not be motivated (1). This means the
productivity of the employees will fall (1). Therefore, the business
may experience problems meeting customer orders (1).
Accept any other appropriate response. Answers that list more
than one disadvantage with no explanation will be awarded a
maximum of 1 mark.""",
"""
Award 1 mark for identification of an advantage, plus 2
further marks for explaining this advantage up to a total of 3
marks.
Retained profit is a cheap source of finance (1). This is because
interest is not paid on retained profit (1). Therefore, the business
would not experience an increase in costs (1).
Retained profit does not dilute the ownership of the business (1).
This is because the owner(s) keep full control of all decisions made
since no new shares are issued (1). This may lead to decisions
being made quicker in the business (1).
Accept any other appropriate response. Answers that list more
than one advantage with no explanation will be awarded a
maximum of 1 mark.
"""
]

In [12]:
structure_edexcel_business_ms_prompt1 = """
Format the Mark Scheme Text supplied below according to the following YAML structure:

marking_criteria:
	- Make a list for each criteria that earns a mark
example_answers:
	-
marking_notes:
	-
total_marks:
	- 

Do not invent anything, extract and format the data only. If there is no information to fall into a YAML section, do not include that section.

Mark Scheme Text:
{mark_scheme_text}
"""

In [13]:
llm=ChatOpenAI(model_name="gpt-4o")

response = run_prompt_chain(
    prompt_template=structure_edexcel_business_ms_prompt1,
    llm=llm,
    mark_scheme_text=edexcel_business_studies_mark_scheme_texts[3]
)

In [14]:
print(response["prompt_chain_response"])

```yaml
marking_criteria:
  - Award 1 mark for identification of a disadvantage, plus 2 further marks for explaining this disadvantage, up to a total of 3 marks.
example_answers:
  - One disadvantage is that the employees will want to leave the business (1). This means the business may have to find new employees (1). This may increase the recruitment costs of the business (1).
  - The employees may not be motivated (1). This means the productivity of the employees will fall (1). Therefore, the business may experience problems meeting customer orders (1).
marking_notes:
  - Accept any other appropriate response. Answers that list more than one disadvantage with no explanation will be awarded a maximum of 1 mark.
total_marks:
  - 3
```


In [15]:
print(response["prompt_chain_token_costing"]["total_cost"])

0.0035250000000000004
